In [ ]:
from data_functions import his_usage_team
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd

{ 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}
{'Chris Paul': 'SAS',"De'Aaron Fox": 'SAS', "Devin Vassell": 'SAS',"Harrison Barnes": 'SAS'}

player_names = {"Payton Pritchard": 'BOS'}
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


player_df, current_defense_stat = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)



for player, df in player_df.items():
    print(player)


    from sklearn.linear_model import Lasso
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import train_test_split
    import numpy as np

    df_X = df.drop(columns=['PTS','Date','Matchup','Team','Home/Away_game','W/L', 'Away', 'season', 'TEAM', 'season_defense'])



    scaler = StandardScaler()
    X = scaler.fit_transform(df_X)

    y = df['PTS']  # Replace with your actual target


    # Define the grid search parameters for Lasso (L1 regularization)
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1, 10]  # Different levels of regularization strength
    }

    # Perform grid search
    grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='r2')
    grid_search.fit(X, y)

    # Print best parameters
    print("Best alpha:", grid_search.best_params_)

    X = pd.DataFrame(X, columns=df_X.columns)  # Convert back to DataFrame


    # Get the best alpha
    best_alpha = grid_search.best_params_['alpha']

    # Fit Lasso with the best alpha
    best_lasso = Lasso(alpha=best_alpha)
    best_lasso.fit(X, y)


    # Get selected (non-zero) feature indices
    selected_features = X.columns[best_lasso.coef_ != 0]
    print("Selected features:", selected_features)

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # display(y_test)

    timestamp = int(pd.Timestamp('2024-12-31').timestamp())
    train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
    test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp

    X_train = train_data[selected_features]
    y_train = train_data['FGM']
    X_test = test_data[selected_features]
    y_test = test_data['FGM']



    # Reduce X to selected features only
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]


    # Retrain Lasso only on selected features
    final_lasso = Lasso(alpha=best_alpha)
    final_lasso.fit(X_train_selected, y_train)

    # Predict on the test set using the reduced feature set
    y_pred = final_lasso.predict(X_test_selected)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("Final RMSE using selected features:", rmse)


In [ ]:
from data_functions import his_usage_team

player_names = {
    "Jayson Tatum": "BOS",
    # # "Nikola Jokic": "DEN",
    # "Jamal Murray": "DEN",
    # "Jaylen Brown": "BOS",
    # "Derrick White": "BOS",
    # "Payton Pritchard": "BOS",
    # "Michael Porter Jr.": "DEN",
    # "Russell Westbrook": "DEN",
    # "Christian Braun": "DEN",
    # "Al Horford": "BOS",
    # # "Julian Strawther": "DEN",
    # "Sam Hauser": "BOS",
    # "Zeke Nnaji": "DEN",
    # "Luke Kornet": "BOS"
}
date_list = ["2023-24","2024-25"]
# usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
stats_path = {
    'usage_path':'D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv',
    'catch_shoot':"D:/nba_tracking_data_csv/nba_csv_{date}/catch_shoot_content.csv",
    'drives':"D:/nba_tracking_data_csv/nba_csv_{date}/drives_content.csv",
    'elbow_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/elbow_touch_content.csv",
    'paint_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/paint_touch_content.csv",
    'passing':"D:/nba_tracking_data_csv/nba_csv_{date}/passing_content.csv",
    'pullup':"D:/nba_tracking_data_csv/nba_csv_{date}/pullup_content.csv",
    'shooting_efficiency':"D:/nba_tracking_data_csv/nba_csv_{date}/shooting_efficiency_content.csv",
    'touches':"D:/nba_tracking_data_csv/nba_csv_{date}/touches_content.csv",
    'tracking_post_ups_content':"D:/nba_tracking_data_csv/nba_csv_{date}/tracking_post_ups_content.csv"
}
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
player_df, current_defense_stat = his_usage_team(player_names, date_list, stats_path, player_base_path, defense_base_path)

for player, df in player_df.items():
    display(player)
    display(df.head(5))

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

def select_features(player_names, date_list, usage_path, player_base_path, defense_base_path, target):
    player_df, _ = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)
    
    selected_features_dict = {}
    
    max_features_player = None
    max_features = 0

    for player, df in player_df.items():
        df_X = df.drop(columns=[target, 'Date', 'Matchup', 'Team', 'Home/Away_game', 'W/L', 'Away', 'season', 'TEAM', 'season_defense'])
        
        # Apply StandardScaler to scale the features
        scaler = StandardScaler()
        X = scaler.fit_transform(df_X)
        y = df[target]  # Target variable
        
        # Grid search parameters for Lasso
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
        
        # Use GridSearchCV to find the best alpha
        grid_search = GridSearchCV(Lasso(max_iter=50000), param_grid, cv=5, scoring='r2')  # Increase max_iter here
        grid_search.fit(X, y)
        
        # Get the best alpha and fit Lasso
        best_alpha = grid_search.best_params_['alpha']
        best_lasso = Lasso(alpha=best_alpha, max_iter=50000)  # Ensure enough iterations for convergence
        best_lasso.fit(X, y)
        
        # Select non-zero coefficient features
        X = pd.DataFrame(X, columns=df_X.columns)
        selected_features = X.columns[best_lasso.coef_ != 0].tolist()
        
        # Store selected features
        selected_features_dict[player] = selected_features
        
        # Track the player with the most features
        if len(selected_features) > max_features:
            max_features = len(selected_features)
            max_features_player = player

    # If a player has no selected features, assign the features of the player with the most features
    for player in selected_features_dict:
        if not selected_features_dict[player]:  # If empty
            selected_features_dict[player] = selected_features_dict.get(max_features_player, [])

    return selected_features_dict



player_names = {
    "Devin Booker": "PHX",
    "Anthony Edwards": "MIN",
    "Kevin Durant": "PHX",
    "Naz Reid": "MIN",
    "Julius Randle": "MIN",
    "Bradley Beal": "PHX",
    "Bol Bol": "PHX",
    "Donte DiVincenzo": "MIN",
    "Jaden McDaniels": "MIN",
    "Nick Richards": "PHX",
    "Mike Conley": "MIN"
}
date_list = ["2022-23","2023-24","2024-25"]
stats_path = {
    'usage_path':'D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv',
    'catch_shoot':"D:/nba_tracking_data_csv/nba_csv_{date}/catch_shoot_content.csv",
    'drives':"D:/nba_tracking_data_csv/nba_csv_{date}/drives_content.csv",
    'elbow_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/elbow_touch_content.csv",
    'paint_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/paint_touch_content.csv",
    'passing':"D:/nba_tracking_data_csv/nba_csv_{date}/passing_content.csv",
    'pullup':"D:/nba_tracking_data_csv/nba_csv_{date}/pullup_content.csv",
    'shooting_efficiency':"D:/nba_tracking_data_csv/nba_csv_{date}/shooting_efficiency_content.csv",
    'touches':"D:/nba_tracking_data_csv/nba_csv_{date}/touches_content.csv",
    'tracking_post_ups_content':"D:/nba_tracking_data_csv/nba_csv_{date}/tracking_post_ups_content.csv"
}
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"

feature_dic = select_features(player_names, date_list, stats_path, player_base_path, defense_base_path,'FGA')

for player, features in feature_dic.items():
    # print(player)
    features = feature_dic[player] 
    print(player,':',features)


In [53]:
# features for shai
# 'MIN_x', 'FGM', 'FG%', '3PA', 'OREB', 'REB', 'RANK'

from feature_function import  select_features
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path, his_usage_team
import pandas as pd
import numpy as np
from IPython.display import display
from scipy.stats import linregress
import sys



# player_names = {"Payton Pritchard": 'BOS'}
player_names = {
    # "Cade Cunningham": "WAS",
    # "Jordan Poole": "WAS",
    # "Malik Beasley": "DET",
    # "Jalen Duren": "DET",
    # "Tobias Harris": "DET",
    # "Alex Sarr": "WAS",
    # "Corey Kispert": "WAS",
    # "Khris Middleton": "WAS",
    # "Ausar Thompson": "DET",
    # "Dennis Schroder": "DET",
    # "Kyshawn George": "WAS",
    # "Tim Hardaway Jr.": "DET",
    # "Richaun Holmes": "WAS",
    # "Marcus Smart": "WAS",
    # "Isaiah Stewart": "DET",
    # "Luka Doncic": "LAL",
    # "Giannis Antetokounmpo": "MIL",
    # "Damian Lillard": "MIL",
    # "Austin Reaves": "LAL",
    # "Kyle Kuzma": "MIL",
    # "Dalton Knecht": "LAL",
    # "Brook Lopez": "MIL",
    # "Gabe Vincent": "LAL",
    # "Dorian Finney-Smith": "LAL",
    # "Taurean Prince": "MIL",
    # "Jarred Vanderbilt": "LAL",
    # "Paolo Banchero": "ORL",
    # "Franz Wagner": "ORL",
    # "Zion Williamson": "NOP",
    # "CJ McCollum": "NOP",
    # "Trey Murphy III": "NOP",
    # "Cole Anthony": "ORL",
    # "Kentavious Caldwell-Pope": "ORL",
    # "Yves Missi": "NOP",
    # "Cam Thomas": "BKN",
    # "Coby White": "CHI",
    # "Cameron Johnson": "BKN",
    # "Kevin Huerter": "BKN",
    # "Matas Buzelis": "CHI",
    # "D'Angelo Russell": "BKN",
    # "Nikola Vucevic": "CHI",
    # "Tre Jones": "BKN",
    # "Zach Collins": "BKN",
    # "Nic Claxton": "BKN",
    # "Ziaire Williams": "BKN",
    # "Keon Johnson": "BKN",
    # "Day'Ron Sharpe": "BKN",
    "Stephen Curry": "GSW",
    "Zach LaVine": "SAC",
    "DeMar DeRozan": "SAC",
    "Jimmy Butler": "GSW",
    "Malik Monk": "SAC",
    "Domantas Sabonis": "SAC",
    "Keegan Murray": "SAC",
    "Moses Moody": "GSW",
    "Buddy Hield": "GSW",
    "Quinten Post": "GSW",
    "Draymond Green": "GSW",
    "Keon Ellis": "SAC"
}



date_list = ["2022-23","2023-24","2024-25"]
stats_path = {
    'usage_path':'D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv',
    'catch_shoot':"D:/nba_tracking_data_csv/nba_csv_{date}/catch_shoot_content.csv",
    'drives':"D:/nba_tracking_data_csv/nba_csv_{date}/drives_content.csv",
    'elbow_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/elbow_touch_content.csv",
    'paint_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/paint_touch_content.csv",
    'passing':"D:/nba_tracking_data_csv/nba_csv_{date}/passing_content.csv",
    'pullup':"D:/nba_tracking_data_csv/nba_csv_{date}/pullup_content.csv",
    'shooting_efficiency':"D:/nba_tracking_data_csv/nba_csv_{date}/shooting_efficiency_content.csv",
    'touches':"D:/nba_tracking_data_csv/nba_csv_{date}/touches_content.csv",
    'tracking_post_ups_content':"D:/nba_tracking_data_csv/nba_csv_{date}/tracking_post_ups_content.csv"
}
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


def prediction(player_names: dict, date_list: list, stats_path, player_base_path, defense_base_path, schedule_base_path,selected_feature_target, prediction_target):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list, stats_path, player_base_path, defense_base_path)
    fga_prediction_results = {}

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    import pandas

    # this is the function that will be used to select the features for the model prints out the player and the best features to use
    feature_dic = select_features(player_names, date_list, stats_path, player_base_path, defense_base_path,selected_feature_target)




    for player, team in player_names.items():
        # Get schedule data for the player's team
        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # Get player-specific prediction data

        df = fga_prediction_data[player]

        features = feature_dic[player] 


        features = [feature for feature in features if feature in df.columns]
        
        if not features:
            print(f"No valid features found for {player} for predicting {prediction_target}")
            continue

        # print(features)
        target = prediction_target

        timestamp = int(pd.Timestamp('2025-02-26').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]
        test_data = df[df['Date_in_Seconds'] >= timestamp]

        # display(test_data)
        if not features:  # If feature_columns is an empty list
            print(f"Skipping training: No selected {target} features.")
            sys.exit(1) 

        X_train = train_data[features].fillna(0)
        y_train = train_data[target].fillna(0)
        X_test = test_data[features].fillna(0)
        y_test = test_data[target].fillna(0)

        # print(f"X_train shape: {X_train.shape}")
        # print(f"y_train shape: {y_train.shape}")
        # print(X_train.head())  # View first few rows
        # print(y_train.head())  # View first few rows

        # print(features)  # Check selected feature names
        # print(train_data.columns)  # See available columns



        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)


        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        # print(f"{player}, MAE: {mae}, RMSE: {rmse}")

        # EWMA calculation for minutes
        alpha = 0.2
        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()
        last_actual = df['MIN_x'].iloc[-1]
        last_smoothed = df['EWMA_MIN'].iloc[-1]
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = round(next_value, 2)


        # Exclude columns
        exclude_features = ['RANK', 'OffRtg', 'W', 'L', 
            'DefRtg','NetRtg', 'AST%', 'AST/TO', 'ASTRatio',
            'OREB%', 'DREB%', 'REB%', 'TOV%' , 'eFG%', 'TS%', 'PACE',
            'POSS', 'TEAM', 'PIE']
        
        exclude_features_schedule = ['home_away', 'schedule_team', 'DATE', 'location', 'season_defense']

        # Get defensive stats for the scheduled team
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)

        # df_defense = df_defense.merge(schedule_df, left_on='TEAM', right_on = 'schedule_team', how='outer', suffixes=('', '_DROP'))
        # df_defense = df_defense.drop(columns=[col for col in df_defense.columns if col.endswith('_DROP')])
        # df_defense= df_defense.fillna(0)


        # display(df_for_schedule)

        


        last_season = df_defense["season_defense"].iloc[-1]
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, exclude_features]

        # display(df_for_schedule)


        first_team = schedule_df['schedule_team'].iloc[0]


        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]


        schedule_values = {feature: df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, feature].values[0] 
                for feature in exclude_features if feature in df_for_schedule.columns}


        
        rolling_features = [col for col in features  if col not in exclude_features]
        # display(rolling_features)

        # df = df[df['Date_in_Seconds'] >= timestamp]
        # # display(df)

        for col in rolling_features:
            df[f'{col}'] = df[col].rolling(window=20).mean().fillna(0).astype(int)

        df_last_rolling = df.iloc[[-1]][[f'{col}' for col in rolling_features]]

        df_last_rolling = df_last_rolling.reset_index(drop=True)

        for value in features:
            if value in exclude_features:
                # print(f'This is the value {value}')
                df_last_rolling[value] = schedule_values.get(value)
                # df_last_rolling[value] = schedule_df[value].iloc[0]

        for value in features:
            if value in exclude_features_schedule:
                # print(f'This is the value {value}')
                # df_last_rolling[value] = schedule_values.get(value)
                df_last_rolling[value] = schedule_df[value].iloc[0]        



        df_last_rolling = df_last_rolling.reindex(columns=features)

        X_future = df_last_rolling

        # display(X_future.head(10))
        # Step 1: Calculate mean and standard deviation of the target variable (PTS, REB, etc.)
        mean_target = y_train.mean()
        std_target = y_train.std()

        # Step 2: Define reasonable bounds (e.g., within 3 standard deviations)
        lower_bound = mean_target - 3 * std_target
        upper_bound = mean_target + 3 * std_target

        

        # future predictions happens here
        future_predictions = model.predict(X_future).astype('int')

        future_predictions = np.clip(future_predictions, lower_bound, upper_bound).astype('int')



        # creating rolling mean average
        df[f"Rolling_Mean_{target}"] = df[target].rolling(window=20).mean()
        df[f"Rolling_Std_{target}"] = df[target].rolling(window=20).std()
        df[f"Rolling_CV_{target}"] = df[f"Rolling_Std_{target}"] / df[f"Rolling_Mean_{target}"]

        rounded_future_prediction = abs(future_predictions[0])

        # print(player)
        # display(X_future.head(10))
        # print(player, target, rounded_future_prediction)
        # display(df[[f"Rolling_CV_{target}"]].tail(1))

        if pd.isna(df[f"Rolling_CV_{target}"].iloc[-1]) or np.isinf(df[f"Rolling_CV_{target}"].iloc[-1]):
            df.loc[df.index[-1], f"Rolling_CV_{target}"] = 0

        rolling_cv = df[f"Rolling_CV_{target}"].iloc[-1]
        highest_cv_seen = df[f"Rolling_CV_{target}"].max()

        cv_fluctuate = rolling_cv * rounded_future_prediction

        if cv_fluctuate > rounded_future_prediction:
            cv_low_prediction = abs(cv_fluctuate - rounded_future_prediction)

        cv_low_prediction = abs(rounded_future_prediction- cv_fluctuate)
        cv_high_prediction = rounded_future_prediction + cv_fluctuate

        player_prediction = f"{cv_low_prediction.astype('int')} to {rounded_future_prediction}"

        

        if rolling_cv > 1:
            confidence_score = max(0, 1 - (rolling_cv / highest_cv_seen))
        else:
            confidence_score = 1 - rolling_cv  # More stability → Higher confidence

        confidence_score_percentage = round(confidence_score * 100, 2)


        lower_bound, upper_bound = cv_low_prediction, rounded_future_prediction




        # Get last 10 games
        recent_games = df[target].tail(2)
        


        # Fit a linear regression (x = game number, y = points)
        slope, intercept, r_value, p_value, std_err = linregress(range(len(recent_games)), recent_games)

        long_term_cv = df["PTS"].rolling(10).std() / df["PTS"].rolling(10).mean()

        # Set dynamic base threshold (scaled by long-term CV)
        base_threshold = max(0.2, min(0.6, 0.3 + 0.2 * long_term_cv.iloc[-1]))

        # Compute dynamic middle threshold (adjusted for rolling CV)
        middle_threshold = max(0.2, min(0.8, base_threshold * (1 + rolling_cv)))
        
        # Check if the slope is close to zero (i.e., in the middle)
        if -middle_threshold <= slope <= middle_threshold:
            trend_status = "stable"
        elif slope > 0:
            trend_status = "trending up"
        else:
            trend_status = "trending down"




        ##### this is for safebet column ############
        import math
        # Step 1: Calculate the midpoint of the range
        midpoint = (lower_bound + upper_bound) / 2
        midpoint = math.floor(midpoint)
    
        
        # Step 2: Adjust the prediction based on confidence score
        if confidence_score_percentage > 60:
            # High confidence - stick closer to midpoint
            if trend_status == "trending up":
                # Trending up - lean towards the higher end
                exact_point = round(midpoint)
            elif trend_status == "trending down":
                # Trending down - lean towards the lower end
                exact_point = lower_bound
            else:
                # Stable - pick midpoint or the closest round number
                exact_point = round(midpoint)
        else:
            # Low confidence - lean more conservatively towards the edges
            if trend_status == "trending up":
                # Trending up - lean towards the higher end
                exact_point = round(midpoint + 1)  # Slight bias to upper end
            elif trend_status == "trending down":
                # Trending down - lean towards the lower end
                exact_point = round(midpoint - 1)  # Slight bias to lower end
            else:
                # Stable - pick midpoint but be cautious (lean lower)
                exact_point = round(midpoint - 1)

        exact_point = int(exact_point)

        if exact_point == -1:
            exact_point = 0

        




        fga_prediction_results[player] = [team,player_prediction,confidence_score_percentage,exact_point]
        if target == 'FGA':
            target = 'PTS'
        
        
        df_results = pd.DataFrame.from_dict(fga_prediction_results, orient='index', columns=['team',target,f'confidence_level_{target}' ,  f'middlebet_{target}'])
        # Reset index and rename it properly
        df_results.reset_index(inplace=True)
        df_results.rename(columns={'index': 'Player'}, inplace=True)
        from datetime import datetime
        today_date = datetime.today().strftime('%Y-%m-%d')
        df_results.to_csv(f'{target}_output_{today_date}.csv', index=False)
        # display(df_results)


    return   df_results

results_reb = prediction(player_names, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'REB','REB')
results_ast = prediction(player_names, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'AST','AST')
results_pts = prediction(player_names, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'PTS','FGA')
results_3pm = prediction(player_names, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'3PM','3PM')
results_pts = results_pts.rename(columns={'FGA': 'PTS'})



# display(results_reb)
# display(results_ast)
# display(results_pts)
# display(results_3pm)

# df_merged = results_reb.merge(results_ast, on='Player', suffixes=('_reb', '_ast')) \
#     .merge(results_pts, on='Player', suffixes=('_pts','extra_pts')) \
#     .merge(results_3pm, on='Player', suffixes=('_pts','_3pm'))


# display(df_merged)


# df_merged['total_points'] = df_merged[['middlebet_PTS', 'middlebet_AST', 'middlebet_REB']].sum(axis=1)
# # df_merged = df_merged.rename(columns={'FGA': 'PTS'})

# display(df_merged[['Player','total_points']])


Player data missing for Quinten Post on 2022-23, skipping.
Skipping merge for Quinten Post on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2023-24, skipping.
Skipping merge for Quinten Post on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2022-23, skipping.
Skipping merge for Quinten Post on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2023-24, skipping.
Skipping merge for Quinten Post on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2022-23, skipping.
Skipping merge for Quinten Post on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2023-24, skipping.
Skipping merge for Quinten Post on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2022-2

c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.584e-01, tolerance: 1.004e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.149e-01, tolerance: 2.243e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

Player data missing for Quinten Post on 2022-23, skipping.
Skipping merge for Quinten Post on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2023-24, skipping.
Skipping merge for Quinten Post on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2022-23, skipping.
Skipping merge for Quinten Post on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2023-24, skipping.
Skipping merge for Quinten Post on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2022-23, skipping.
Skipping merge for Quinten Post on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2023-24, skipping.
Skipping merge for Quinten Post on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Quinten Post on 2022-2

In [77]:

display(results_reb)
display(results_ast)
# display(results_pts)
# display(results_3pm)

df_merged = results_reb.merge(results_ast, on=['Player', 'team'], suffixes=('_reb', '_ast')) \
    .merge(results_pts, on=['Player', 'team'], suffixes=('_pts','extra_pts')) \
    .merge(results_3pm, on=['Player', 'team'], suffixes=('_pts','_3pm'))

['Player', 'REB', 'confidence_level_REB', 'middlebet_REB', 'AST', 'confidence_level_AST', 'middlebet_AST', 'PTS', 'confidence_level_PTS', 'middlebet_PTS', '3PM', 'confidence_level_3PM', 'middlebet_3PM']

# print(df_merged.columns)


df_merged['middlebet_PTS+REB'] = 0000

df_merged['PTS+REB'] = df_merged[['middlebet_PTS', 'middlebet_REB']].sum(axis=1)
df_merged['confidence_level_PTS+REB']=  round(df_merged[['confidence_level_PTS','confidence_level_REB']].sum(axis=1)/2,2)
# df_merged = df_merged.rename(columns={'FGA': 'PTS'})
# display(df_merged)
# display(df_merged[['Player','confidence_level_PTS+REB']])

,Player,team,REB,confidence_level_REB,middlebet_REB
0,Stephen Curry,GSW,0 to 1,44.80,0
1,Zach LaVine,SAC,1 to 3,55.59,1
2,DeMar DeRozan,SAC,1 to 3,46.09,1
3,Jimmy Butler,GSW,1 to 5,20.25,4
4,Malik Monk,SAC,0 to 2,33.50,0
5,Domantas Sabonis,SAC,8 to 14,58.67,10
6,Keegan Murray,SAC,3 to 5,67.98,3
7,Moses Moody,GSW,1 to 3,40.08,1
8,Buddy Hield,GSW,0 to 2,43.91,0
9,Quinten Post,GSW,1 to 4,45.94,3


,Player,team,AST,confidence_level_AST,middlebet_AST
0,Stephen Curry,GSW,3 to 6,53.05,3
1,Zach LaVine,SAC,2 to 5,41.45,4
2,DeMar DeRozan,SAC,1 to 3,50.98,1
3,Jimmy Butler,GSW,2 to 5,56.88,4
4,Malik Monk,SAC,6 to 10,63.66,6
5,Domantas Sabonis,SAC,5 to 11,46.47,7
6,Keegan Murray,SAC,0 to 1,2.07,0
7,Moses Moody,GSW,0 to 1,3.32,0
8,Buddy Hield,GSW,0 to 1,15.61,1
9,Quinten Post,GSW,1 to 4,33.33,3


In [79]:
import random
import pandas as pd

def create_parlays(player_data, num_groups, players_per_group, min_confidence=7):
    # List of all possible player data points (player, stat, and range)
    all_players = []
    # print(player_data)

    for idx, row in player_data.iterrows():
        player_name = row['Player']
        stats = ['REB', 'AST', 'PTS', '3PM']
        team = row['team']
        for stat in stats:
            stat_range = row[stat]
            confidence = row[f'confidence_level_{stat}']
            safebet = row[f'middlebet_{stat}']

            # Only add players with confidence level at or above the minimum
            if confidence >= min_confidence:
                all_players.append((player_name,team,stat, stat_range, confidence, safebet))
    
    # Sort by confidence level (highest first)
    all_players.sort(key=lambda x: x[3], reverse=True)
    
    # Create the parlays based on num_groups and players_per_group
    parlays = []
    
    # Make a copy of all_players to work with
    available_players = all_players.copy()
    
    for _ in range(num_groups):
        parlay = []
        # Try to select players_per_group high-confidence players
        for _ in range(players_per_group):
            if available_players:
                # Get a player from the sorted list (highest confidence first)
                selected_player = available_players.pop(0)
                parlay.append(selected_player)
            else:
                # If we run out of players, reshuffle and start again
                available_players = all_players.copy()
                random.shuffle(available_players)  # Add some randomness for variety
                if available_players:
                    selected_player = available_players.pop(0)
                    parlay.append(selected_player)
        
        if parlay:  # Only add non-empty parlays
            parlays.append(parlay)

    # Create a list of DataFrames for each parlay
    parlays_df = []
    group_confidence_scores = []
    for idx, parlay in enumerate(parlays, 1):
        parlay_name = f"Parlay {idx}"
        parlay_data = {
            'Player': [],
            'team': [],
            'Stat': [],
            'Stat Range': [],
            'Confidence': [],
            'middlebet': []
        }
        total_confidence = 0
        for player,team ,stat, stat_range, confidence, safebet in parlay:
            parlay_data['Player'].append(player)
            parlay_data['team'].append(team)
            parlay_data['Stat'].append(stat)
            parlay_data['Stat Range'].append(stat_range)
            parlay_data['Confidence'].append(confidence)
            parlay_data['middlebet'].append(safebet)
            total_confidence += confidence
        
        # Calculate the average confidence score for this parlay
        avg_confidence = total_confidence / len(parlay) if parlay else 0
        group_confidence_scores.append((parlay_name, avg_confidence))
        
        # Convert to DataFrame
        df = pd.DataFrame(parlay_data)
        parlays_df.append(df)
    
    return parlays_df, group_confidence_scores


player_data = df_merged
# For very high confidence parlays
parlays_high, scores_high = create_parlays(player_data, num_groups=5, players_per_group=6, min_confidence=90)

# For moderate confidence parlays
parlays_medium, scores_medium = create_parlays(player_data, num_groups=5, players_per_group=3, min_confidence=6)



print("Confidence scores for each group:")
for group, score in scores_high:
    print(f"{group}: {score:.2f}")


# Display the DataFrames for each parlay and their confidence scores
for (parlay_name, score), parlay_df in zip(scores_high, parlays_high):
    print("\n")
    print(f"{parlay_name}: total confidence score: {score:.2f}")
    display(parlay_df)
    print("\n")

Confidence scores for each group:
Parlay 1: 93.87
Parlay 2: 93.87
Parlay 3: 93.87
Parlay 4: 93.87
Parlay 5: 93.87


Parlay 1: total confidence score: 93.87


,Player,team,Stat,Stat Range,Confidence,middlebet
0,Buddy Hield,GSW,PTS,7 to 8,93.32,7
1,Keon Ellis,SAC,PTS,5 to 6,91.56,5
2,Zach LaVine,SAC,PTS,15 to 16,96.73,15
3,Zach LaVine,SAC,PTS,15 to 16,96.73,15
4,Buddy Hield,GSW,PTS,7 to 8,93.32,7
5,Keon Ellis,SAC,PTS,5 to 6,91.56,5






Parlay 2: total confidence score: 93.87


,Player,team,Stat,Stat Range,Confidence,middlebet
0,Zach LaVine,SAC,PTS,15 to 16,96.73,15
1,Keon Ellis,SAC,PTS,5 to 6,91.56,5
2,Buddy Hield,GSW,PTS,7 to 8,93.32,7
3,Keon Ellis,SAC,PTS,5 to 6,91.56,5
4,Buddy Hield,GSW,PTS,7 to 8,93.32,7
5,Zach LaVine,SAC,PTS,15 to 16,96.73,15






Parlay 3: total confidence score: 93.87


,Player,team,Stat,Stat Range,Confidence,middlebet
0,Buddy Hield,GSW,PTS,7 to 8,93.32,7
1,Keon Ellis,SAC,PTS,5 to 6,91.56,5
2,Zach LaVine,SAC,PTS,15 to 16,96.73,15
3,Buddy Hield,GSW,PTS,7 to 8,93.32,7
4,Keon Ellis,SAC,PTS,5 to 6,91.56,5
5,Zach LaVine,SAC,PTS,15 to 16,96.73,15






Parlay 4: total confidence score: 93.87


,Player,team,Stat,Stat Range,Confidence,middlebet
0,Buddy Hield,GSW,PTS,7 to 8,93.32,7
1,Zach LaVine,SAC,PTS,15 to 16,96.73,15
2,Keon Ellis,SAC,PTS,5 to 6,91.56,5
3,Buddy Hield,GSW,PTS,7 to 8,93.32,7
4,Keon Ellis,SAC,PTS,5 to 6,91.56,5
5,Zach LaVine,SAC,PTS,15 to 16,96.73,15






Parlay 5: total confidence score: 93.87


,Player,team,Stat,Stat Range,Confidence,middlebet
0,Buddy Hield,GSW,PTS,7 to 8,93.32,7
1,Keon Ellis,SAC,PTS,5 to 6,91.56,5
2,Zach LaVine,SAC,PTS,15 to 16,96.73,15
3,Buddy Hield,GSW,PTS,7 to 8,93.32,7
4,Zach LaVine,SAC,PTS,15 to 16,96.73,15
5,Keon Ellis,SAC,PTS,5 to 6,91.56,5


In [45]:
import random
import pandas as pd



# Convert to DataFrame
player_data = df_merged

def create_parlays(player_data, num_groups, players_per_group):
    # List of all possible player data points (player, stat, and range)
    all_players = []

    for idx, row in player_data.iterrows():
        player_name = row['Player']
        stats = ['REB', 'AST', 'PTS', '3PM', 'PTS+REB']
        for stat in stats:
            stat_range = row[stat]
            confidence = row[f'confidence_level_{stat}']
            safebet = row[f'middlebet_{stat}']

            all_players.append((player_name, stat, stat_range, confidence, safebet))
                

    # Randomize the order of the player data
    random.shuffle(all_players)

    # Create the parlays based on num_groups and players_per_group
    parlays = []
    for _ in range(num_groups):
        parlay = []
        for _ in range(players_per_group):
            if all_players:
                parlay.append(all_players.pop())
        parlays.append(parlay)

    # Create a list of DataFrames for each parlay
    parlays_df = []
    group_confidence_scores = []
    for idx, parlay in enumerate(parlays, 1):
        parlay_name = f"Parlay {idx}"
        parlay_data = {
            'Player': [],
            'Stat': [],
            'Stat Range': [],
            'Confidence': [],
            'middlebet': []
        }
        total_confidence = 0  # Variable to calculate the total confidence score
        for player, stat, stat_range, confidence, safebet in parlay:
            parlay_data['Player'].append(player)
            parlay_data['Stat'].append(stat)
            parlay_data['Stat Range'].append(stat_range)
            parlay_data['Confidence'].append(confidence)
            parlay_data['middlebet'].append(safebet)
            total_confidence += confidence
        
        # Calculate the average confidence score for this parlay
        avg_confidence = total_confidence / len(parlay)
        group_confidence_scores.append((parlay_name, avg_confidence))
        
        # Convert to DataFrame
        df = pd.DataFrame(parlay_data)
        
        # df["middlebet"] = df["middlebet"].astype(int)


        # Apply middlebet correction **only to the DataFrame, not the entire list**
        # df.loc[(df["Stat"] == "PTS+REB") & (df["middlebet"] == 0), "middlebet"] = None

        parlays_df.append(df)



    
    return parlays_df, group_confidence_scores

# Example usage
num_groups = 5 # Number of parlays you want to generate
players_per_group = 7  # Number of players in each parlay

parlays_df, group_confidence_scores = create_parlays(player_data, num_groups, players_per_group)


# Display the confidence scores for each group
print("Confidence scores for each group:")
for group, score in group_confidence_scores:
    print(f"{group}: {score:.2f}")


# Display the DataFrames for each parlay and their confidence scores
for (parlay_name, score), parlay_df in zip(group_confidence_scores, parlays_df):
    print("\n")
    print(f"{parlay_name}: total confidence score: {score:.2f}")
    display(parlay_df)
    print("\n")



Confidence scores for each group:
Parlay 1: 49.02
Parlay 2: 38.19
Parlay 3: 32.88
Parlay 4: 49.66
Parlay 5: 52.11


Parlay 1: total confidence score: 49.02


,Player,Stat,Stat Range,Confidence,middlebet
0,Malik Monk,PTS+REB,12,54.15,0
1,Stephen Curry,3PM,3 to 7,47.59,4
2,Keon Ellis,AST,0 to 1,12.44,0
3,Keon Ellis,PTS,6 to 7,91.56,6
4,Malik Monk,PTS,10 to 14,74.80,12
5,Stephen Curry,AST,2 to 5,53.05,2
6,DeMar DeRozan,3PM,0 to 3,9.52,0






Parlay 2: total confidence score: 38.19


,Player,Stat,Stat Range,Confidence,middlebet
0,Quinten Post,3PM,0 to 1,28.50,0
1,Domantas Sabonis,PTS+REB,17,58.82,0
2,Keegan Murray,3PM,1 to 3,36.49,1
3,Buddy Hield,PTS+REB,7,68.61,0
4,Zach LaVine,3PM,1 to 5,33.07,2
5,Keegan Murray,AST,0 to 1,2.07,0
6,Keon Ellis,REB,0 to 2,39.79,0






Parlay 3: total confidence score: 32.88


,Player,Stat,Stat Range,Confidence,middlebet
0,Jimmy Butler,3PM,0 to 1,29.42,0
1,Malik Monk,REB,0 to 2,33.50,0
2,Keon Ellis,3PM,0 to 2,18.28,0
3,Domantas Sabonis,PTS,6 to 11,58.96,7
4,Moses Moody,AST,0 to 1,3.32,0
5,Draymond Green,3PM,0 to 0,19.31,0
6,DeMar DeRozan,PTS,10 to 15,67.35,10






Parlay 4: total confidence score: 49.66


,Player,Stat,Stat Range,Confidence,middlebet
0,Keon Ellis,PTS+REB,6,65.68,0
1,Moses Moody,3PM,0 to 2,26.56,0
2,Zach LaVine,REB,1 to 3,55.59,1
3,Domantas Sabonis,3PM,0 to 2,21.08,0
4,Stephen Curry,REB,0 to 1,44.80,0
5,Stephen Curry,PTS+REB,12,57.75,0
6,Moses Moody,PTS,6 to 9,76.16,7






Parlay 5: total confidence score: 52.11


,Player,Stat,Stat Range,Confidence,middlebet
0,Zach LaVine,AST,1 to 4,41.45,1
1,Stephen Curry,PTS,12 to 18,70.70,12
2,Quinten Post,PTS+REB,8,48.93,0
3,Quinten Post,AST,0 to 0,33.33,0
4,Malik Monk,AST,1 to 2,63.66,1
5,Draymond Green,PTS+REB,6,52.94,0
6,Jimmy Butler,PTS,5 to 10,53.74,6


In [46]:
import random
import pandas as pd
import numpy as np

# Convert to DataFrame
player_data = df_merged

def weighted_random_choice(choices, weights, k):
    return random.choices(choices, weights=weights, k=k)

def create_parlays_high(player_data, num_groups, players_per_group):
    # List of all possible player data points (player, stat, and range)
    all_players = []
    confidence_levels = []

    for idx, row in player_data.iterrows():
        player_name = row['Player']
        stats = ['REB', 'AST', 'PTS', '3PM', 'PTS+REB']
        for stat in stats:
            stat_range = row[stat]
            confidence = row[f'confidence_level_{stat}']
            safebet = row[f'middlebet_{stat}']

            all_players.append((player_name, stat, stat_range, confidence, safebet))
            confidence_levels.append(confidence)  # Store confidence levels

    parlays = []
    group_confidence_scores = []
    
    for _ in range(num_groups):
        if not all_players:
            break  # Stop if we run out of players

        # Weighted selection of players
        selected_players = weighted_random_choice(all_players, confidence_levels, players_per_group)

        # Remove selected players from the pool
        for player in selected_players:
            index = all_players.index(player)
            del all_players[index]
            del confidence_levels[index]  # Keep lists synchronized

        # Organize selected players into DataFrame
        parlay_data = {
            'Player': [p[0] for p in selected_players],
            'Stat': [p[1] for p in selected_players],
            'Stat Range': [p[2] for p in selected_players],
            'Confidence': [p[3] for p in selected_players],
            'middlebet': [p[4] for p in selected_players],
        }
        df = pd.DataFrame(parlay_data)

        # Calculate the group's average confidence
        avg_confidence = np.mean(parlay_data['Confidence'])
        group_confidence_scores.append((f"Parlay {len(parlays)+1}", avg_confidence))

        parlays.append(df)

    return parlays, group_confidence_scores


# Define the number of parlays and players per parlay
num_groups = 2  # Example: Create 3 parlays
players_per_group = 8 # Example: Each parlay consists of 4 players

# Call the function
parlays, confidence_scores = create_parlays_high(df_merged, num_groups, players_per_group)

# Display results
for i, (df, (parlay_name, avg_conf)) in enumerate(zip(parlays, confidence_scores), 1):
    print(f"\n{parlay_name} (Avg Confidence: {avg_conf:.2f})")
    display(df)


ValueError: ('Jimmy Butler', '3PM', '0 to 1', 29.42, 0) is not in list

In [ ]:
print("Confidence scores for each group:")
for group, score in group_confidence_scores:
    print(f"{group}: {score:.2f}")


# Display the DataFrames for each parlay and their confidence scores
for (parlay_name, score), parlay_df in zip(group_confidence_scores, parlays_df):
    print("\n")
    print(f"{parlay_name}: total confidence score: {score:.2f}")
    display(parlay_df)
    print("\n")

In [ ]:
import random
import pandas as pd



# Convert to DataFrame
player_data = df_merged

def create_parlays(player_data, num_groups, players_per_group):
    # List of all possible player data points (player, stat, and range)
    all_players = []

    for idx, row in player_data.iterrows():
        player_name = row['Player']
        stats = ['REB', 'AST', 'PTS', '3PM']
        for stat in stats:
            stat_range = row[stat]
            confidence = row[f'confidence_level_{stat}']
            safebet = row[f'safebet_{stat}']
            all_players.append((player_name, stat, stat_range, confidence, safebet))

    # Randomize the order of the player data
    random.shuffle(all_players)

    # Create the parlays based on num_groups and players_per_group
    parlays = []
    for _ in range(num_groups):
        parlay = []
        for _ in range(players_per_group):
            if all_players:
                parlay.append(all_players.pop())
        parlays.append(parlay)

    # Create a list of DataFrames for each parlay
    parlays_df = []
    group_confidence_scores = []
    for idx, parlay in enumerate(parlays, 1):
        parlay_name = f"Parlay {idx}"
        parlay_data = {
            'Player': [],
            'Stat': [],
            'Stat Range': [],
            'Confidence': [],
            'Safebet': []
        }
        total_confidence = 0  # Variable to calculate the total confidence score
        for player, stat, stat_range, confidence, safebet in parlay:
            parlay_data['Player'].append(player)
            parlay_data['Stat'].append(stat)
            parlay_data['Stat Range'].append(stat_range)
            parlay_data['Confidence'].append(confidence)
            parlay_data['Safebet'].append(safebet)
            total_confidence += confidence
        
        # Calculate the average confidence score for this parlay
        avg_confidence = total_confidence / len(parlay)
        group_confidence_scores.append((parlay_name, avg_confidence))
        
        parlays_df.append(pd.DataFrame(parlay_data))
    
    return parlays_df, group_confidence_scores

# Example usage
num_groups = 7  # Number of parlays you want to generate
players_per_group = 5  # Number of players in each parlay

parlays_df, group_confidence_scores = create_parlays(player_data, num_groups, players_per_group)


# Display the confidence scores for each group
print("Confidence scores for each group:")
for group, score in group_confidence_scores:
    print(f"{group}: {score:.2f}")


# Display the DataFrames for each parlay and their confidence scores
for (parlay_name, score), parlay_df in zip(group_confidence_scores, parlays_df):
    print("\n")
    print(f"{parlay_name}: total confidence score: {score:.2f}")
    display(parlay_df)
    print("\n")



In [ ]:
from feature_function import fga_prediction


SAS = {'Chris Paul': 'SAS',"De'Aaron Fox": 'SAS', "Devin Vassell": 'SAS',"Harrison Barnes": 'SAS'}
OKC = { 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}

player_names = SAS
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"

results = fga_prediction(player_names, date_list, usage_path, player_base_path, defense_base_path, schedule_base_path)


for player, fga_predictions in results.items():
    print(fga_predictions)


In [ ]:
def build_data_path(base_path, **Kwargs):
    # Replace placeholders with actual values
    for key, value in Kwargs.items():
        base_path=base_path.replace(f"{{{key}}}", str(value))
    return base_path

base_path = "/data/{year}/{month}/{day}/file.txt"
kwargs = {'year': 2025, 'month': '02', 'day': '10'}
year = 2025
build_data_path(base_path, year=year)

In [ ]:
import pandas as pd

schedule_df = pd.read_csv("D:/nba_scheduled_csv/schedule_csv_2025/OKC_schedule_content.csv")

schedule_df

In [ ]:
def fga_prediction(player_names: dict, date_list: list, usage_path, player_base_path, defense_base_path, schedule_base_path):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list,usage_path,player_base_path, defense_base_path)

    fga_prediction_results = {}

    for player, team in player_names.items():

        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # print(schedule_df)

    for player, df in fga_prediction_data.items():

        from sklearn.model_selection import train_test_split
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.metrics import mean_squared_error
        from sklearn.model_selection import cross_val_score
        from sklearn.linear_model import LinearRegression
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        import numpy as np



        features = ['PACE', 'team_pace', 'USG', 'DefRtg','MIN_x', 'home_away', 'Date_in_Seconds','OffRtg', 'team_offrtg']
        target = 'FGA'


        # Continue with your existing operations
        timestamp = int(pd.Timestamp('2024-12-31').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
        test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp


        X_train = train_data[features]
        y_train = train_data[target]
        X_test = test_data[features]
        y_test = test_data[target]


        model = LinearRegression()
        model.fit(X_train,y_train)

        y_pred = model.predict(X_test)

        ###### predicting Minutes ##########
        alpha = 0.2

        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()

        last_actual = df['MIN_x'].iloc[-1]  # Last known FGA
        last_smoothed = df['EWMA_MIN'].iloc[-1]  # Last smoothed value
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = next_value.round(2)


        ##################################################################

        last_season = df_defense["season_defense"].iloc[-1] 
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, ['TEAM','PACE', 'DefRtg', 'OffRtg']]

        # display(df_for_schedule)


        first_team = schedule_df['schedule_team'].iloc[0]
        # print(first_team)
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]
        # print(schedule_team_result)

        schedule_defrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'DefRtg'].values[0]

        schedule_pace = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'PACE'].values[0]

        schedule_offrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'OffRtg'].values[0]

        # print("dfrtg:",schedule_defrtg)
        # print("pace",schedule_pace)
        # print("offrtg",schedule_offrtg)

        # This is to turn the first date in schedule into seconds
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9

        # This is to turn the home and away games into a 1 or a zero -> away is 1 and anything is zero
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)


        

        # display(schedule_df)





        X_future = pd.DataFrame({
        # 'EWMA_FGA_2': [next_value], 
        'PACE':[schedule_pace], 
        'team_pace':[df['team_pace'].iloc[-1]], 
        'USG':[df['USG'].iloc[-1]],
        'DefRtg':[schedule_defrtg],
        'MIN_x':[next_value], 
        'home_away':[schedule_df['home_away'].iloc[0]],
        'Date_in_Seconds':[schedule_df['Date_in_Seconds'].iloc[0]],
        'OffRtg':[df['OffRtg'].iloc[-1]],
        'team_offrtg': [schedule_offrtg]
        })

        display(X_future)

        future_predictions = model.predict(X_future)



        # print("this is future",future_predictions)

        fga_prediction_results.update({player:future_predictions[0]})

        
    



    return fga_prediction_results

In [ ]:
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path
import pandas as pd
import numpy as np
from IPython.display import display


def his_usage_team(player_names: dict, date_list: list, usage_path,player_base_path,defense_base_path):
    current_player_dic = {}

    for player, team in player_names.items():
        current_player_frames =[]

        for date in date_list:
            usage_path =build_data_path(usage_path,date=date)
            usage_data = pd.read_csv(usage_path)

            #merging player and defense dat into one
            merged_data, current_defense_df = his_player_defense_data(player_base_path,defense_base_path,player,date)

            #adding season to usage_data
            usage_data['season'] = date

            #Getting the player usage percentage for usage data and adding to merge
            player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
            merged_data['USG'] = player_usage

            #adding the current player team pace
            team_stat = current_defense_df.loc[current_defense_df['TEAM'] == team, 'PACE'].values[0]
            merged_data["team_pace"] = team_stat

            # adding current player team OffRtg
            team_offrtg = current_defense_df.loc[current_defense_df['TEAM'] == team, 'OffRtg'].values[0]
            merged_data["team_offrtg"] = team_offrtg

            team_poss = current_defense_df.loc[current_defense_df['TEAM'] == team, 'POSS'].values[0]
            merged_data["team_poss"] = team_poss
            
            # Exclude rows where the TEAM column matches the given team
            merged_data = merged_data[merged_data['TEAM'] != team]


            # merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

            # Turn date into seconds
            merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
            merged_data = merged_data.sort_values(by="Date_in_Seconds")


            # Turn Home/Away game into 1 and 0
            merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
            # Dropping duplicates
            merged_data = merged_data.drop_duplicates()
            
            # Append the DataFrame for this date to the player's list
            current_player_frames.append(merged_data)

        # Combine all dates for the current player into one DataFrame
        current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)


    return current_player_dic, current_defense_df


player_names = { 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


def fga_prediction(player_names: dict, date_list: list, usage_path, player_base_path, defense_base_path, schedule_base_path):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)
    fga_prediction_results = {}

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    from sklearn.preprocessing import StandardScaler




    for player, team in player_names.items():
        # Get schedule data for the player's team
        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # Get player-specific prediction data

        df = fga_prediction_data[player]





        
        # Model training and prediction code
        features = ['PACE', 'team_pace', 'USG', 'DefRtg', 'MIN_x', 'home_away', 'Date_in_Seconds', 'OffRtg', 'team_offrtg']
        target = 'FGA'

        timestamp = int(pd.Timestamp('2024-12-31').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]
        test_data = df[df['Date_in_Seconds'] >= timestamp]

        X_train = train_data[features]
        y_train = train_data[target]
        X_test = test_data[features]
        y_test = test_data[target]

        # Initialize Scaler
        scaler = StandardScaler()
        # Transform Data
        scaled_data_x = scaler.fit_transform(X_train)
        scaled_data_y = scaler.fit_transform(y_train.values.reshape(-1, 1))

        X_train = pd.DataFrame(scaled_data_x, columns=X_train.columns)     
        y_train = pd.DataFrame(scaled_data_y, columns=['features'])   

        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)


        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"{player}, MAE: {mae}, RMSE: {rmse}")

        # EWMA calculation for minutes
        alpha = 0.2
        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()
        last_actual = df['MIN_x'].iloc[-1]
        last_smoothed = df['EWMA_MIN'].iloc[-1]
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = round(next_value, 2)

        # Get defensive stats for the scheduled team
        last_season = df_defense["season_defense"].iloc[-1]
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, ['TEAM', 'PACE', 'DefRtg', 'OffRtg']]

        first_team = schedule_df['schedule_team'].iloc[0]
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]
        schedule_defrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'DefRtg'].values[0]
        schedule_pace = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'PACE'].values[0]
        schedule_offrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'OffRtg'].values[0]

        # Convert schedule dates to seconds
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)

        # Create future prediction dataframe
        X_future = pd.DataFrame({
            'PACE': [schedule_pace],
            'team_pace': [df['team_pace'].iloc[-1]],
            'USG': [df['USG'].iloc[-1]],
            'DefRtg': [schedule_defrtg],
            'MIN_x': [next_value],
            'home_away': [schedule_df['home_away'].iloc[0]],
            'Date_in_Seconds': [schedule_df['Date_in_Seconds'].iloc[0]],
            'OffRtg': [df['OffRtg'].iloc[-1]],
            'team_offrtg': [schedule_offrtg]
        })

        future_predictions = model.predict(X_future)
        fga_prediction_results[player] = future_predictions[0]

    return fga_prediction_results

results = fga_prediction(player_names, date_list, usage_path, player_base_path, defense_base_path, schedule_base_path)


for player, fga_predictions in results.items():
    print(fga_predictions)
   

            

            

            

            



            


In [ ]:
import data_functions as data_functions
import pandas as pd
import numpy as np
import importlib
importlib.invalidate_caches()
from data_functions import his_player_defense_data, current_player_defense_data

#"D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
#"D:\nba_defense_csv_current\defense_csv_2024-25\all_quarter_defense_content.csv"
#"D:\nba_scheduled_csv\schedule_csv_2025\ATL_schedule_content.csv"

player_names = {'Shai Gilgeous-Alexander':'OKC', 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC'}
date_list = ["2022-23","2023-24","2024-25"]
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
# usage_data = f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv"



current_player_dic = {}

for player, team in player_names.items():
  current_player_frames = []

  for date in date_list:
    usage_data = pd.read_csv(f"D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv")  

    # use current_player function to to merge datat
    merged_data, current_player_defense = his_player_defense_data(player_base_path,defense_base_path,player,date)

    # Add the season column to usage_data
    usage_data['season'] = date
    
    # Get the player's usage percentage from the usage data
    # (Assumes that there is exactly one matching row)
    player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
    merged_data['USG'] = player_usage


    # adding current player team pace
    team_stat = current_player_defense.loc[current_player_defense['TEAM'] == team, 'PACE'].values[0]
    merged_data["team_pace"] = team_stat

    # display(current_player_defense)

    # adding current player team OffRtg
    team_offrtg = current_player_defense.loc[current_player_defense['TEAM'] == team, 'OffRtg'].values[0]
    merged_data["team_offrtg"] = team_offrtg

    team_poss = current_player_defense.loc[current_player_defense['TEAM']== team, 'POSS'].values[0]
    merged_data["team_poss"] = team_poss

    # Exclude rows where the TEAM column matches the given team
    merged_data = merged_data[merged_data['TEAM'] != team]
    # display(merged_data.head(5))

    # merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

    # Turn date into seconds
    merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
    merged_data = merged_data.sort_values(by="Date_in_Seconds")

    # Turn Home/Away game into 1 and 0
    merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
    # Dropping duplicates
    merged_data = merged_data.drop_duplicates()
    
    # Append the DataFrame for this date to the player's list
    current_player_frames.append(merged_data)

  # Combine all dates for the current player into one DataFrame
  current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)



specific_player = 'Shai Gilgeous-Alexander'
for player, df in current_player_dic.items():

    print(f"\nData for {player}:")
    df['FGA_rolling_3'] = df['FGA'].rolling(window=3).mean()



    alpha = 0.2

    df['EWMA_FGA'] = df['FGA'].ewm(span=(2/(1-alpha)-1), adjust=False).mean()

    df['EWMA_FGA_2'] = df['FGA'].ewm(span=(2/alpha - 1), adjust=False).mean()

    # alpha = 0.2  # Example smoothing factor
    df['Exp_smooth'] = 21  # Initialize column

    for i in range(1, len(df)):
        df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']

    df_act = df

    # display(df_act)

    # display(df)
    df = df.head(149)

    value_fga_list = []
    moving_average_list = []
    next_next_value_list = []
    date_list = []
    
    for value_fga, moving_average, date in zip(df['FGA'],df['EWMA_FGA_2'],df['Date']):
       value_fga_list.append(value_fga)
       moving_average_list.append(moving_average)
       #print("this is actual:",value_fga, "this is last_predicted",moving_average)
       next_next_value = alpha * value_fga + (1 - alpha) * moving_average
       next_next_value_list.append(next_next_value)
       date_list.append(date)
       #print("this is next:",next_next_value)
       dataframe_dic= {'Date':date_list,'Actual_FGA': value_fga_list, 'Moving_average':moving_average_list, 'Next':next_next_value_list}
       dataframe = pd.DataFrame(dataframe_dic)
      #  dataframe['Next'] = dataframe['Next'].shift(1)

    # display(dataframe)


    # Predict the next (11th) value
    last_actual = df['FGA'].iloc[-1]  # Last known FGA
    last_smoothed = df['EWMA_FGA_2'].iloc[-1]  # Last smoothed value
    next_value = alpha * last_actual + (1 - alpha) * last_smoothed


    # Print and add the prediction
    print(f"Predicted value: {next_value}")

    # display(df)

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error



    features = ['PACE', 'team_pace', 'USG', 'DefRtg','MIN_x', 'home_away', 'Date_in_Seconds','OffRtg', 'team_offrtg']
    target = 'FGA'


     # Continue with your existing operations
    timestamp = int(pd.Timestamp('2024-12-31').timestamp())
    train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
    test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp


    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]


    model = LinearRegression()
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    # print(y_pred)

     # Compare predictions with actual points
    predicted_vs_actual = pd.DataFrame({
        "Date": test_data['Date'],  
        "Matchup": test_data['Matchup'],  
        "Predicted Points": y_pred.round(1), 
        "Actual Points": y_test}).reset_index(drop=True)
    
    display(predicted_vs_actual)

    X_future = pd.DataFrame({
    # 'EWMA_FGA_2': [next_value], 
    'PACE':[df['PACE'].iloc[-1]], 
    'team_pace':[df['team_pace'].iloc[-1]], 
    'USG':[df['USG'].iloc[-1]],
    'DefRtg':[df['DefRtg'].iloc[-1]],
    'MIN_x':[30.09], 
    'home_away':[1],
    'Date_in_Seconds':[1738368000],
    'OffRtg':[df['OffRtg'].iloc[-1]],
    'team_offrtg': [df['team_offrtg'].iloc[-1]]
    })
    
    future_predictions = model.predict(X_future)

    print("this is future",future_predictions)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"MAE: {mae}, RMSE: {rmse}")

    import numpy as np

    # Drop NaN values (first row of 'Next' is NaN due to shift)
    rmse_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute RMSE
    rmse = np.sqrt(((rmse_df['Actual_FGA'] - rmse_df['Next']) ** 2).mean())

    print("RMSE:", rmse)


    # Drop NaN values (due to shift)
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Total number of valid comparisons
    total = len(comparison_df)

    # Count occurrences
    higher_count = (comparison_df['Next'] > comparison_df['Actual_FGA']).sum()
    lower_count = (comparison_df['Next'] < comparison_df['Actual_FGA']).sum()
    equal_count = (comparison_df['Next'] == comparison_df['Actual_FGA']).sum()

    # Calculate percentages
    higher_percent = (higher_count / total) * 100
    lower_percent = (lower_count / total) * 100
    equal_percent = (equal_count / total) * 100

    # Print results
    print(f"Percentage of predictions HIGHER than actual: {higher_percent:.2f}%")
    print(f"Percentage of predictions LOWER than actual: {lower_percent:.2f}%")
    print(f"Percentage of predictions EQUAL to actual: {equal_percent:.2f}%")


    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Filter cases where Next is higher than Actual
    higher_cases = comparison_df[comparison_df['Next'] > comparison_df['Actual_FGA']]
    higher_difference_avg = (higher_cases['Next'] - higher_cases['Actual_FGA']).mean()

    # Filter cases where Next is lower than Actual
    lower_cases = comparison_df[comparison_df['Next'] < comparison_df['Actual_FGA']]
    lower_difference_avg = (lower_cases['Actual_FGA'] - lower_cases['Next']).mean()

    # Print results
    print(f"Average difference when prediction is HIGHER: {higher_difference_avg:.2f}")
    print(f"Average difference when prediction is LOWER: {lower_difference_avg:.2f}")

    # Drop NaN values
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute Mean Error (ME)
    mean_error = (comparison_df['Next'] - comparison_df['Actual_FGA']).mean()

    # Print result
    print(f"Mean Error (ME): {mean_error:.2f}")

    